In [1]:
from utils import *
from skimage import measure
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


1. Считываем данные и приводим к единой системе координат

In [2]:
# Размер воксельного пространства
VOXEL_GRID_SIZE = 256

In [3]:
# Получим пути к исходным файлам
files = [f for f in os.listdir("data") if f.endswith('.x')]

In [4]:
meshes = []
for f in files:
    print(f)
    filepath = os.path.abspath(f'data\\{f}')
    data = parse_x_file(filepath)

    # Переведём координаты точек из локальной системы координат в систему координат камеры
    data['vertices'] = apply_transformation(data.get('vertices'), data.get('frame_matrix'))
    
    # Путь к текстуре
    data['texture_path'] = os.path.abspath(f'data\\{f.rstrip(".x")}.bmp')
    meshes.append(data)

teapot_1.x
num_vertices: 200310
num_faces: 379304
num_uvs: 200310
teapot_2.x
num_vertices: 190307
num_faces: 358169
num_uvs: 190307


In [5]:
# Конфертируем все меши в PointCloud
pcds_raw = [vertices_to_pcd(m.get('vertices')) for m in meshes]
# Считаем нормали для PointCloud
pcds_down = [get_normals_to_pcd(pcd) for pcd in pcds_raw]

# Посмотрим как взаиморасположены меши
o3d.visualization.draw_geometries(pcds_down)

2. Инициализируем воксельное пространство

In [6]:
# Сбор всех вершин в единую структуру
all_vertices = np.vstack([mesh["vertices"] for mesh in meshes])

In [7]:
# Границы сцены
min_bound = np.min(all_vertices, axis=0)
max_bound = np.max(all_vertices, axis=0)
scene_size = max_bound - min_bound

In [8]:
# Размер одного вокселя
voxel_size = scene_size / VOXEL_GRID_SIZE

In [9]:
# Инициализация скалярного поля и весов
D = np.zeros((VOXEL_GRID_SIZE + 1, VOXEL_GRID_SIZE + 1, VOXEL_GRID_SIZE + 1), dtype=np.float32)
W = np.zeros_like(D)

In [10]:
# Сохраняем параметры для дальнейших преобразований
voxel_space = {
    "min_bound": min_bound,
    "max_bound": max_bound,
    "scene_size": scene_size,
    "voxel_size": voxel_size,
    "D": D,
    "W": W,
    "grid_size": VOXEL_GRID_SIZE
}

3. Объединяем меши воксельным методом

In [247]:
# Вычислим интегральную функцию растояния и весовую функцию для каждого вокселя
for mesh in meshes:
    integrate_mesh_to_voxel_grid(mesh, voxel_space)

Faces: 100%|██████████| 379k/379k [02:53<00:00, 2.18k triangles/s] 
Faces: 100%|██████████| 358k/358k [02:48<00:00, 2.12k triangles/s] 


In [248]:
safe_D = np.copy(D)

In [249]:
safe_A = np.copy(safe_D)

In [256]:
# Marching cubes ищет поверхность, где D == 0
verts, faces, normals, values = measure.marching_cubes(safe_D, level=0.0, spacing=(voxel_space["voxel_size"]))

In [257]:
# Создадим меш на основе вершин и треугольников
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts)
mesh.triangles = o3d.utility.Vector3iVector(faces)

In [258]:
mesh = mesh.filter_smooth_simple(number_of_iterations=3)

In [259]:
# Пересчитаем нормали
mesh.compute_vertex_normals()
mesh.compute_triangle_normals()
mesh.normalize_normals()

TriangleMesh with 276201 points and 552476 triangles.

In [262]:
# Посмотрим на результат объединения
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

4. Сохраняем полученный меш

In [261]:
write_mesh_to_x(mesh, 'output_mesh.x', save_normals=False, save_texture=False)

Vertices: 100%|██████████| 276k/276k [00:01<00:00, 143k points/s] 
Faces: 100%|██████████| 552k/552k [00:01<00:00, 320k triangles/s] 
